**Текущие задачи**:
- Реализовать проверку новых статей для tproger
- Фактически для проверки начала работы, нам достаточно знать лишь `parse_time`
- Выгружать датасет из памяти после того, как он был считан. Составлять новый датасет, выводить в csv и конкатенировать сами csv.
- Функция обновления views_num (и других показателей)
- Функция поиска новых статей и составления по ним записей: сравниваем текущую дату и дату последней статьи, запускаем поочередное обращение к ресурсам
- Оформить корпус: csv + zip с файлами полных текстов
- Однотипное оформление столбцов
- Однотипное оформление файлов

Исходные данные собраны посредством `wget` и уже предобработаны. Однако со временем на ресурсах публикуются новые статьи, а старые статьи продолжают набирать просмотры. Сбор и обновление соответствующей информации помогают учесть появление новых технологий.

Чтобы не создавать повышенную нагрузку на серверы ресурсов и не вызывать их подозрений, необходимо их опрашивать поочередно. Все необходимые методы содержатся внутри класса `Parser`, а оркестровку парсеров проводит объект класса `ParserComposer`.

In [1]:
# standard library
import datetime
import re
import uuid
import pickle

# conections
import requests
from requests.exceptions import ConnectionError

# parsers libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

# data science 
import pandas as pd
from tqdm.notebook import tqdm, trange

sources = {
    'tproger':
        {
            'base_url': 'https://tproger.ru',
            'first_page_url': 'https://tproger.ru/page/1/'},
    'habr':
        {
            'base_url': 'https://habr.com'
        }
}

for source in sources:
    sources[source]['df'] = pd.read_csv(f'../../DATASETS/{source}.csv.zip',
                               index_col=0,
                               parse_dates=['post_time', 'parse_time'])

In [4]:
class Parser:
    def __init__(self, source):
        self.df = pd.read_csv(f'../../DATASETS/{source}.csv.zip',
                         index_col=0,
                         parse_dates=['post_time', 'parse_time'])
    
    def delta_dates_days(self):
        today_date = datetime.datetime.now()
        last_parse_date = self.df.post_time.max()
        today_date = today_date.replace(tzinfo=last_parse_date.tz)
        delta_dates = today_date - last_parse_date
        return delta_dates.days
    
    def check_new_articles(self):
        '''
        Проверяет, появились ли на ресурсе новые публикации,
        и если появились, заносит их в план на скачивание.
        '''
        pass

    def update_corpora(self):
        '''
        Скачивает новые статьи и данные о них для датафрейма.
        '''
        pass
    
    def update_views(self):
        '''
        Обновляет число просмотров у скачанных статей.
        '''
        pass
    
    def concat_csv(self):
        '''
        Соединяет старый csv и тот, что получен из новых данных.
        '''
        pass
    

class ParserComposer:
    def __init__(self):
        pass
    
    def scheduler(self):
        '''
        Сортирует задачи таким образом, чтобы
        равномерно распределить запросы к внешним ресурсам
        '''
        pass
    
    def normal_pipeline(self):
        # check_new_articles()
        # планировщик
        # update_corpora
        # update_vies для последних из новых
        pass

In [5]:
tproger_parser = Parser('tproger')

In [6]:
tproger_parser.delta_dates_days()

14

In [7]:

    source = 'tproger'
    article_urls = []
    page_url = sources[source]['first_page_url']
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    post_links = soup.find_all('a', {'class':'article-link'})
    article_urls = [url['href'] for url in post_links]

In [ ]:
get

In [ ]:
for i in trange(1, 81):    
    for url in tqdm(urls):
        if url in data[source]:
            continue
        else:
            try:
                get_data_from_tproger_article(data, source, url)
            except ConnectionError:
                time.sleep(60)

In [101]:
sources

{'tproger': {'base_url': 'https://tproger.ru',
  'first_page_url': 'https://tproger.ru/page/1'},
 'habr': {'base_url': 'https://habr.com'}}

In [148]:
page_new_articles_urls

{'https://tproger.ru/articles/5-sposobov-pobedit-vygoranie-u-programmistov/',
 'https://tproger.ru/articles/6-sposobov-bolshe-uznat-pro-algoritmy/',
 'https://tproger.ru/articles/8-jazykov-programmirovanija-dlja-android-razrabotchika/',
 'https://tproger.ru/articles/chto-novogo-v-java-16-i-kak-na-jeto-otreagirovali-razrabotchiki/',
 'https://tproger.ru/articles/chto-pochitat-nachinajushhemu-testirovshhiku/',
 'https://tproger.ru/articles/kak-programmistu-sdelat-horoshij-ux/',
 'https://tproger.ru/articles/kakoj-samyj-bezopasnyj-messendzher-dlja-polzovatelej/',
 'https://tproger.ru/articles/kejs-habib-vs-makgregor-ili-raspoznavanie-lic-na-python/',
 'https://tproger.ru/articles/metanavyki-chto-jeto-takoe-i-zachem-ih-uchit/',
 'https://tproger.ru/articles/moja-istorija-v-it-ot-programmirovanija-s-otcom-do-mezhdunarodnogo-proekta/',
 'https://tproger.ru/articles/opyt-v-menedzhmente-ili-v-it-chto-vazhnee-kogda-hochesh-stat-prodzhekt-menedzherom/',
 'https://tproger.ru/articles/osnovnoj-nab

In [8]:
all_new_urls = set()
page_count = 1


True

In [10]:
def check_new_articles_tproger(source='tproger'):
    all_new_urls, page_count, source = set(), 1, source   
    while True:   
        page_url = sources[source]['first_page_url'].replace('1', str(page_count))
        page = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        article_links = soup.find_all('a', {'class':'article-link'})
        urls = {url['href'] for url in article_links}
        old_urls = set(sources[source]['df'].index.to_list())
        new_urls = urls - old_urls
        all_new_urls = all_new_urls | new_urls
        if len(new_urls) < len(urls):
            print(f"{len(all_new_urls)} new urls for {source}.")
            break
        else:
            page_count += 1

{'https://tproger.ru/articles/6-sposobov-bolshe-uznat-pro-algoritmy/', 'https://tproger.ru/articles/moja-istorija-v-it-ot-programmirovanija-s-otcom-do-mezhdunarodnogo-proekta/', 'https://tproger.ru/articles/pochemu-vam-stoit-vybrat-node-js-dlja-razrabotki-servernyh-prilozhenij/', 'https://tproger.ru/articles/reaktivnye-formy-v-angular/', 'https://tproger.ru/articles/5-sposobov-pobedit-vygoranie-u-programmistov/', 'https://tproger.ru/articles/kak-programmistu-sdelat-horoshij-ux/', 'https://tproger.ru/articles/kakoj-samyj-bezopasnyj-messendzher-dlja-polzovatelej/', 'https://tproger.ru/books/stoit-prochitat-obzor-knigi-medzhui-mehdi-nepreryvnoe-razvitie-api-pravilnye-reshenija-v-izmenchivom-tehnologicheskom-landshafte/', 'https://tproger.ru/articles/metanavyki-chto-jeto-takoe-i-zachem-ih-uchit/', 'https://tproger.ru/articles/stoit-prochitat-vospominanija-introvertov-izmenivshih-mir/', 'https://tproger.ru/articles/postojannyj-kalendar-mechta-programmista/', 'https://tproger.ru/articles/tes

In [140]:
a = set([1]).update(set([2]))
a

In [128]:
 - test

{'https://tproger.ru/articles/5-sposobov-pobedit-vygoranie-u-programmistov/',
 'https://tproger.ru/articles/6-sposobov-bolshe-uznat-pro-algoritmy/',
 'https://tproger.ru/articles/8-jazykov-programmirovanija-dlja-android-razrabotchika/',
 'https://tproger.ru/articles/chto-novogo-v-java-16-i-kak-na-jeto-otreagirovali-razrabotchiki/',
 'https://tproger.ru/articles/chto-pochitat-nachinajushhemu-testirovshhiku/',
 'https://tproger.ru/articles/kak-programmistu-sdelat-horoshij-ux/',
 'https://tproger.ru/articles/kakoj-samyj-bezopasnyj-messendzher-dlja-polzovatelej/',
 'https://tproger.ru/articles/kejs-habib-vs-makgregor-ili-raspoznavanie-lic-na-python/',
 'https://tproger.ru/articles/metanavyki-chto-jeto-takoe-i-zachem-ih-uchit/',
 'https://tproger.ru/articles/moja-istorija-v-it-ot-programmirovanija-s-otcom-do-mezhdunarodnogo-proekta/',
 'https://tproger.ru/articles/opyt-v-menedzhmente-ili-v-it-chto-vazhnee-kogda-hochesh-stat-prodzhekt-menedzherom/',
 'https://tproger.ru/articles/osnovnoj-nab